# Simplest use case

## Running a model from the config file

Setting up the `dolphin` ecosystem:

- Install `dolphin`, `lenstronomy`, and the required dependencies.
- Create an input/output directory for `dolphin`, we are using "../io_directory_example" in this example.
- Set up these directories inside the input/output directory. Look inside "../io_directory_example" for example. 
    - **data**: contains subdirectories for each lens system with a data and PSF files for each band.
    - **settings**: contains the 'config_{lens_name}.yml' files for each lens system,
    - **logs**: to write the log files from model runs,
    - **outputs**: to save the model outputs,
    - **hpc**: *optional*, contains scripts to submit batch jobs in MPI.
 

   
### Data format

The image data file needs to be in the hdf5 formate with the following datasets:

- `image_data`: reduced and background subtracted image cutout centered at the lens system,
- `background_rms`: background level,
- `exposure_time`: the map of exposure times for each pixel, so that `image_data * exposure_time` is Poisson noise distributed,
- `ra_at_xy_0`: RA of the (0, 0) pixel in the `image_data` cutout,
- `dec_at_xy_0`: Dec of the (0, 0) pixel in the `image_data` cutout,
- `transform_pix2angle`: a transform matrix to map the pixel numbers (x, y) to angles (RA, Dec).

The PSF data file needs to be in the hdf5 format with the following datasets:

- `kernel_point_source`: a pixelated PSF (not required to have the same dimension of `image_data`),
- `psf_error_map`: *optional*, uncertainty in the provided PSF, needs to have the same dimension of `kernel_point_source`. 

### Config file format

The 'config_{lens_name}.yml' file provides you options to set up the lens model. Here is the content of the config file that is used in the example model below.

### create a `Processor` instance and point to the IO directory

In [1]:
from dolphin.processor import Processor

processor = Processor('../io_directory_example/')

### Run a model by calling the  `swim()` method

In [2]:
processor.swim(lens_name='lens_system2', model_id='example', log=False)

Computing the PSO ...
10
20
30
40
50
Max iteration reached! Stopping.
-1.4122667766269474 reduced X^2 of best position
-10146.430656676304 log likelihood
14369 effective number of data points
[{'theta_E': 1.2486720936802873, 'gamma': 2.0, 'e1': -0.010836279872904194, 'e2': 0.0027505856028065155, 'center_x': 0.04507729787488673, 'center_y': 0.012887660458767543}, {'gamma_ext': 1.7430049671250024e-05, 'psi_ext': 0.09800423813706911, 'ra_0': 0, 'dec_0': 0}] lens result
[{'amp': 1, 'n_max': 4, 'beta': 0.10567485171811598, 'center_x': 0.08300082262701894, 'center_y': -0.2608960464929205}] source result
[{'amp': 1, 'R_sersic': 1.1702050258030843, 'n_sersic': 4.0, 'e1': -0.01818066498020119, 'e2': 0.05000250312340953, 'center_x': 0.030223411575640253, 'center_y': -0.06058162182267721}] lens light result
[] point source result
{} special param result
107.9222400188446 time used for  PSO
Computing the PSO ...
10
20
30
40
50
Max iteration reached! Stopping.
-1.330704054776778 reduced X^2 of best

100%|██████████| 100/100 [11:33<00:00,  6.94s/it] 


Computing the MCMC...
Number of walkers =  32
Burn-in iterations:  0
Sampling iterations (in current run): 100
698.7187950611115 time taken for MCMC sampling


## Let's check the output

In [3]:
from dolphin.analysis.output import Output

output = Output('../io_directory_example/')

In [4]:
fig = output.plot_model_overview(lens_name='lens_system2', model_id='example')

TypeError: __init__() got an unexpected keyword argument 'likelihood_mask_list'

We only ran a pre-sampling optimization here and did not perform a MCMC sampling. So, the above model is the optimized model that the MCMC sample can initiate from. The `kwargs_result` dictionary of the pre-sampling optimization step can be accessed through `Output.kwargs_result` after loading an output using the `Output.load_output()` method.

In [ ]:
output.load_output(lens_name='lens_system2', model_id='example')

output.kwargs_result

{'kwargs_lens': [{'theta_E': 1.2326899640213018,
   'gamma': 1.998918057975498,
   'e1': 0.023771532230263397,
   'e2': 0.007322808057297473,
   'center_x': 0.07425521322394245,
   'center_y': -0.21335353495417878},
  {'gamma_ext': 0.004831374500885104,
   'psi_ext': -0.07328085140675819,
   'ra_0': 0,
   'dec_0': 0}],
 'kwargs_source': [{'amp': 1,
   'n_max': 4,
   'beta': 0.1472201512075073,
   'center_x': 0.07919321428442905,
   'center_y': -0.2524963794337973}],
 'kwargs_lens_light': [{'amp': 1,
   'R_sersic': 2.703115633851944,
   'n_sersic': 4.0,
   'e1': -0.11964362032082303,
   'e2': 0.000947148809756073,
   'center_x': 0.024595750787618078,
   'center_y': -0.053359526560273124}],
 'kwargs_ps': [],
 'kwargs_special': {},
 'kwargs_extinction': []}

When necessary, the settings of the model run---that was read out of the 'config_{lens_name}.yml' file---can be accessed through the `output.model_settings` variable.

In [ ]:
output.model_settings

{'system_name': 'lens_system2',
 'band': ['F390W'],
 'pixel_size': 0.04,
 'model': {'lens': ['SPEP', 'SHEAR_GAMMA_PSI'],
  'lens_light': ['SERSIC_ELLIPSE'],
  'source_light': ['SHAPELETS']},
 'lens_option': {'centroid_init': [0.04, -0.04]},
 'lens_light_option': {'fix': {'0': {'n_sersic': 4.0}}},
 'source_light_option': {'n_max': [4]},
 'fitting': {'pso': True,
  'pso_settings': {'num_particle': 20, 'num_iteration': 50},
  'sampling': True,
  'sampler': 'MCMC',
  'mcmc_settings': {'burnin_step': 0,
   'iteration_step': 100,
   'walker_ratio': 2}},
 'numeric_option': {'supersampling_factor': [2]}}